In [2]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")


# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

In [6]:
# Check if the 'theme' column exists
if 'theme' in df.columns:
    # Display unique themes in the 'theme' column
    unique_themes = df['theme'].unique()
    print("Unique themes in the 'theme' column:")
    print(unique_themes)
else:
    print("The 'theme' column does not exist in the dataset.")

Unique themes in the 'theme' column:
['Energie' 'Werk' 'Economie' 'Mobiliteit en Verkeer'
 'Europese instellingen' 'Openbare werken'
 'Lokale overheden en Binnenlands bestuur' 'Integratie en Inburgering'
 'Gelijke kansen' 'Onderwijs en Vorming' 'Cultuur' 'Media' 'Begroting'
 'Jeugdbeleid' 'Onroerend erfgoed' 'Brussel en de Vlaamse Rand'
 'Welzijn en Gezin' 'Gezondheid' 'Wonen' 'Landbouw'
 'Wetenschap en Innovatie' 'Financiën' 'Buitenlands beleid' 'Toerisme'
 'Justitie en Handhaving' 'Natuur en Milieu' 'Vlaamse administratie'
 'Dierenwelzijn' 'Ruimtelijke ordening' 'Oekraïnecrisis'
 'Internationaal ondernemen'
 'Staatshervorming en Verhoudingen binnen de Belgische federale staat'
 'Sport' 'Armoedebeleid' 'Ontwikkelingssamenwerking' 'Sociale economie'
 'Vlaamse Regering' 'Taalgebruik']


In [7]:
# Thema-cluster mapping
theme_merge_map = {
    # Bestuur en Beleid
    "Lokale overheden en Binnenlands bestuur": "Bestuur en Beleid",
    "Vlaamse administratie": "Bestuur en Beleid",
    "Staatshervorming en Verhoudingen binnen de Belgische federale staat": "Bestuur en Beleid",
    "Vlaamse Regering": "Bestuur en Beleid",

    # Mobiliteit en Infrastructuur
    "Mobiliteit en Verkeer": "Mobiliteit en Infrastructuur",
    "Openbare werken": "Mobiliteit en Infrastructuur",
    "Ruimtelijke ordening": "Mobiliteit en Infrastructuur",

    # Economie en Arbeid
    "Werk": "Economie en Arbeid",
    "Economie": "Economie en Arbeid",
    "Sociale economie": "Economie en Arbeid",
    "Internationaal ondernemen": "Economie en Arbeid",

    # Welzijn en Gezondheid
    "Welzijn en Gezin": "Welzijn en Gezondheid",
    "Gezondheid": "Welzijn en Gezondheid",
    "Armoedebeleid": "Welzijn en Gezondheid",

    # Cultuur en Communicatie
    "Cultuur": "Cultuur en Communicatie",
    "Media": "Cultuur en Communicatie",
    "Onroerend erfgoed": "Cultuur en Communicatie",
    "Taalgebruik": "Cultuur en Communicatie",

    # Onderwijs en Samenleving
    "Onderwijs en Vorming": "Onderwijs en Samenleving",
    "Gelijke kansen": "Onderwijs en Samenleving",
    "Jeugdbeleid": "Onderwijs en Samenleving",
    "Integratie en Inburgering": "Onderwijs en Samenleving",

    # Milieu en Landbouw
    "Natuur en Milieu": "Milieu en Landbouw",
    "Landbouw": "Milieu en Landbouw",
    "Dierenwelzijn": "Milieu en Landbouw",

    # Internationaal Beleid
    "Buitenlands beleid": "Internationaal Beleid",
    "Europese instellingen": "Internationaal Beleid",
    "Ontwikkelingssamenwerking": "Internationaal Beleid",
    "Oekraïnecrisis": "Internationaal Beleid",

    # Overige (apart laten tenzij weinig samples)
    "Financiën": "Financiën",
    "Begroting": "Begroting",
    "Wetenschap en Innovatie": "Wetenschap en Innovatie",
    "Toerisme": "Toerisme",
    "Justitie en Handhaving": "Justitie en Handhaving",
    "Brussel en de Vlaamse Rand": "Brussel en de Vlaamse Rand",
    "Sport": "Sport"
}

# Nieuwe kolom aanmaken met samengevoegde thema's
df["theme_merged"] = df["theme"].map(theme_merge_map).fillna("Onbekend")



In [8]:
print(len(df))

39125


In [13]:
# Remove rows where the 'question' column has 5 or fewer words
df = df[df['question'].apply(lambda x: len(str(x).split()) > 7)]

In [14]:
print(len(df))

32046
